In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import random
from dataclasses import dataclass
from typing import List, Tuple, Dict
import time
from datetime import datetime, timedelta

warnings.filterwarnings('ignore')

In [47]:
@dataclass
class GAHyperparams:
    """Hyperparameter configuration for LSTM model"""
    sequence_length: int
    hidden_size: int
    num_layers: int
    dropout: float
    learning_rate: float
    batch_size: int
    epochs: int

    def to_dict(self) -> Dict:
        return {
            'sequence_length': self.sequence_length,
            'hidden_size': self.hidden_size,
            'num_layers': self.num_layers,
            'dropout': self.dropout,
            'learning_rate': self.learning_rate,
            'batch_size': self.batch_size,
            'epochs': self.epochs
        }

In [48]:
class LSTMCell(nn.Module):
    """LSTM Cell implemented from scratch"""
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Weight matrices for input-to-hidden connections
        self.W_ii = nn.Parameter(torch.randn(hidden_size, input_size))
        self.W_if = nn.Parameter(torch.randn(hidden_size, input_size))
        self.W_ig = nn.Parameter(torch.randn(hidden_size, input_size))
        self.W_io = nn.Parameter(torch.randn(hidden_size, input_size))

        # Weight matrices for hidden-to-hidden connections
        self.W_hi = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.W_hf = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.W_hg = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.W_ho = nn.Parameter(torch.randn(hidden_size, hidden_size))

        # Bias vectors
        self.b_i = nn.Parameter(torch.randn(hidden_size))
        self.b_f = nn.Parameter(torch.randn(hidden_size))
        self.b_g = nn.Parameter(torch.randn(hidden_size))
        self.b_o = nn.Parameter(torch.randn(hidden_size))

        self.init_weights()

    def init_weights(self):
        """Initialize weights using Xavier initialization"""
        std = 1.0 / (self.hidden_size)**0.5
        for weight in self.parameters():
            weight.data.uniform_(-std, std)

    def forward(self, x, hidden):
        h_prev, c_prev = hidden

        # Input gate
        i_t = torch.sigmoid(torch.mm(self.W_ii, x.t()) + torch.mm(self.W_hi, h_prev.t()) + self.b_i.unsqueeze(1))

        # Forget gate
        f_t = torch.sigmoid(torch.mm(self.W_if, x.t()) + torch.mm(self.W_hf, h_prev.t()) + self.b_f.unsqueeze(1))

        # Cell gate (candidate values)
        g_t = torch.tanh(torch.mm(self.W_ig, x.t()) + torch.mm(self.W_hg, h_prev.t()) + self.b_g.unsqueeze(1))

        # Output gate
        o_t = torch.sigmoid(torch.mm(self.W_io, x.t()) + torch.mm(self.W_ho, h_prev.t()) + self.b_o.unsqueeze(1))

        # Update cell state
        c_t = f_t * c_prev.t() + i_t * g_t

        # Update hidden state
        h_t = o_t * torch.tanh(c_t)

        return h_t.t(), c_t.t()

In [49]:
class LSTM(nn.Module):
    """Multi-layer LSTM implemented from scratch"""
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        # Create LSTM layers
        self.lstm_cells = nn.ModuleList()
        for i in range(num_layers):
            layer_input_size = input_size if i == 0 else hidden_size
            self.lstm_cells.append(LSTMCell(layer_input_size, hidden_size))

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        # Output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden=None):
        batch_size, seq_len, _ = x.size()
        if hidden is None:
            hidden = self.init_hidden(batch_size, x.device)

        # Process each time step
        outputs = []
        for t in range(seq_len):
            x_t = x[:, t, :]

            # Process through each LSTM layer
            layer_input = x_t
            new_hidden = []
            for i, lstm_cell in enumerate(self.lstm_cells):
                h_t, c_t = lstm_cell(layer_input, (hidden[i][0], hidden[i][1]))
                new_hidden.append((h_t, c_t))
                layer_input = self.dropout(h_t)

            hidden = new_hidden
            outputs.append(layer_input)

        # Use the last output for prediction
        last_output = outputs[-1]
        prediction = self.fc(last_output)

        return prediction, hidden

    def init_hidden(self, batch_size, device):
        """Initialize hidden states"""
        hidden = []
        for _ in range(self.num_layers):
            h = torch.zeros(batch_size, self.hidden_size, device=device)
            c = torch.zeros(batch_size, self.hidden_size, device=device)
            hidden.append((h, c))
        return hidden

In [50]:
class GeneticAlgorithmOptimizer:
    """Genetic Algorithm for LSTM hyperparameter optimization"""
    def __init__(self, population_size: int = 15, generations: int = 8,
                 mutation_rate: float = 0.15, crossover_rate: float = 0.8):
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate

        # Define hyperparameter ranges optimized for stock prediction
        self.param_ranges = {
            'sequence_length': (20, 90),  # Days to look back
            'hidden_size': (50, 200),     # LSTM hidden units
            'num_layers': (2, 4),         # Number of LSTM layers
            'dropout': (0.1, 0.4),        # Dropout rate
            'learning_rate': (0.0005, 0.01),  # Learning rate
            'batch_size': [16, 32, 64],   # Batch sizes (powers of 2)
            'epochs': (50, 150)           # Training epochs
        }
        self.best_individuals = []  # Track best individuals from each generation

    def create_individual(self) -> GAHyperparams:
        """Create a random individual (hyperparameter set)"""
        return GAHyperparams(
            sequence_length=random.randint(*self.param_ranges['sequence_length']),
            hidden_size=random.randint(*self.param_ranges['hidden_size']),
            num_layers=random.randint(*self.param_ranges['num_layers']),
            dropout=random.uniform(*self.param_ranges['dropout']),
            learning_rate=random.uniform(*self.param_ranges['learning_rate']),
            batch_size=random.choice(self.param_ranges['batch_size']),
            epochs=random.randint(*self.param_ranges['epochs'])
        )

    def initialize_population(self) -> List[GAHyperparams]:
        """Initialize the population"""
        population = []

        # Add some good starting points based on common practices
        good_configs = [
            GAHyperparams(60, 100, 3, 0.2, 0.001, 32, 100),  # Your original config
            GAHyperparams(30, 128, 2, 0.25, 0.002, 64, 80),  # Fast training
            GAHyperparams(90, 64, 4, 0.3, 0.0008, 16, 120),  # Deep model
        ]

        # Add good configs to population
        for config in good_configs[:min(len(good_configs), self.population_size)]:
            population.append(config)

        # Fill rest with random individuals
        while len(population) < self.population_size:
            population.append(self.create_individual())

        return population

    def fitness_function(self, individual: GAHyperparams, predictor) -> float:
        """
        Evaluate fitness of an individual (hyperparameter set)
        Lower fitness is better (we're minimizing validation loss)
        """
        try:
            print(f"Testing: seq_len={individual.sequence_length}, hidden={individual.hidden_size}, "
                  f"layers={individual.num_layers}, lr={individual.learning_rate:.4f}")

            # Prepare data with individual's sequence length
            original_seq_len = predictor.sequence_length
            predictor.sequence_length = individual.sequence_length

            # Get fresh data preparation
            X_train, y_train, X_val, y_val, _ = predictor.prepare_data(
                predictor.raw_data, predictor.feature_cols
            )

            # Create model with individual's hyperparameters
            model = LSTM(
                input_size=X_train.shape[2],
                hidden_size=individual.hidden_size,
                num_layers=individual.num_layers,
                output_size=1,
                dropout=individual.dropout
            ).to(predictor.device)

            # Train with limited epochs for GA speed
            train_epochs = min(individual.epochs, 60)  # Cap epochs for GA speed
            val_loss = self._train_and_evaluate(
                model, individual, X_train, y_train, X_val, y_val, train_epochs
            )

            # Restore original sequence length
            predictor.sequence_length = original_seq_len

            print(f"→ Validation Loss: {val_loss:.6f}")
            return val_loss

        except Exception as e:
            print(f"→ Error: {str(e)}")
            return float('inf')

    def _train_and_evaluate(self, model, params, X_train, y_train, X_val, y_val, epochs):
        """Train model and return validation loss"""
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)

        best_val_loss = float('inf')
        patience = 10
        patience_counter = 0

        # Create data loader for batching
        train_size = X_train.shape[0]

        for epoch in range(epochs):
            model.train()
            total_loss = 0
            num_batches = 0

            # Mini-batch training
            for i in range(0, train_size, params.batch_size):
                end_idx = min(i + params.batch_size, train_size)
                batch_X = X_train[i:end_idx]
                batch_y = y_train[i:end_idx]

                optimizer.zero_grad()
                outputs, _ = model(batch_X)
                loss = criterion(outputs.squeeze(), batch_y)
                loss.backward()

                # Gradient clipping for stability
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                optimizer.step()
                total_loss += loss.item()
                num_batches += 1

            # Validation
            if epoch % 5 == 0:  # Check validation every 5 epochs
                model.eval()
                with torch.no_grad():
                    val_outputs, _ = model(X_val)
                    val_loss = criterion(val_outputs.squeeze(), y_val).item()

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    patience_counter = 0
                else:
                    patience_counter += 1

                # Early stopping for GA efficiency
                if patience_counter >= patience:
                    break

        return best_val_loss

    def selection(self, population: List[GAHyperparams],
                  fitness_scores: List[float]) -> List[GAHyperparams]:
        """Tournament selection"""
        selected = []
        tournament_size = 3

        for _ in range(len(population)):
            # Select random individuals for tournament
            tournament_indices = random.sample(range(len(population)),
                                             min(tournament_size, len(population)))
            tournament_fitness = [fitness_scores[i] for i in tournament_indices]

            # Select best individual from tournament (lowest fitness)
            winner_idx = tournament_indices[tournament_fitness.index(min(tournament_fitness))]
            selected.append(population[winner_idx])

        return selected

    def crossover(self, parent1: GAHyperparams,
                  parent2: GAHyperparams) -> Tuple[GAHyperparams, GAHyperparams]:
        """Uniform crossover"""
        if random.random() > self.crossover_rate:
            return parent1, parent2

        # Create offspring by mixing parameters
        child1 = GAHyperparams(
            sequence_length=parent1.sequence_length if random.random() < 0.5 else parent2.sequence_length,
            hidden_size=parent1.hidden_size if random.random() < 0.5 else parent2.hidden_size,
            num_layers=parent1.num_layers if random.random() < 0.5 else parent2.num_layers,
            dropout=parent1.dropout if random.random() < 0.5 else parent2.dropout,
            learning_rate=parent1.learning_rate if random.random() < 0.5 else parent2.learning_rate,
            batch_size=parent1.batch_size if random.random() < 0.5 else parent2.batch_size,
            epochs=parent1.epochs if random.random() < 0.5 else parent2.epochs
        )

        child2 = GAHyperparams(
            sequence_length=parent2.sequence_length if random.random() < 0.5 else parent1.sequence_length,
            hidden_size=parent2.hidden_size if random.random() < 0.5 else parent1.hidden_size,
            num_layers=parent2.num_layers if random.random() < 0.5 else parent1.num_layers,
            dropout=parent2.dropout if random.random() < 0.5 else parent1.dropout,
            learning_rate=parent2.learning_rate if random.random() < 0.5 else parent1.learning_rate,
            batch_size=parent2.batch_size if random.random() < 0.5 else parent1.batch_size,
            epochs=parent2.epochs if random.random() < 0.5 else parent1.epochs
        )

        return child1, child2

    def mutate(self, individual: GAHyperparams) -> GAHyperparams:
        """Mutate an individual"""
        if random.random() > self.mutation_rate:
            return individual

        # Choose random parameter to mutate
        param_to_mutate = random.choice(list(self.param_ranges.keys()))

        if param_to_mutate == 'sequence_length':
            individual.sequence_length = random.randint(*self.param_ranges['sequence_length'])
        elif param_to_mutate == 'hidden_size':
            individual.hidden_size = random.randint(*self.param_ranges['hidden_size'])
        elif param_to_mutate == 'num_layers':
            individual.num_layers = random.randint(*self.param_ranges['num_layers'])
        elif param_to_mutate == 'dropout':
            individual.dropout = random.uniform(*self.param_ranges['dropout'])
        elif param_to_mutate == 'learning_rate':
            individual.learning_rate = random.uniform(*self.param_ranges['learning_rate'])
        elif param_to_mutate == 'batch_size':
            individual.batch_size = random.choice(self.param_ranges['batch_size'])
        elif param_to_mutate == 'epochs':
            individual.epochs = random.randint(*self.param_ranges['epochs'])

        return individual

    def optimize(self, predictor) -> Tuple[GAHyperparams, float]:
        """Main GA optimization loop"""
        print(f"\nStarting Genetic Algorithm Optimization")
        print(f"Population: {self.population_size}, Generations: {self.generations}")
        print("="*60)

        # Initialize population
        population = self.initialize_population()
        best_individual = None
        best_fitness = float('inf')

        for generation in range(self.generations):
            print(f"\nGeneration {generation + 1}/{self.generations}")
            print("-" * 40)
            start_time = time.time()

            # Evaluate fitness
            fitness_scores = []
            for i, individual in enumerate(population):
                print(f"Individual {i+1}/{len(population)}")
                fitness = self.fitness_function(individual, predictor)
                fitness_scores.append(fitness)

                # Track best individual
                if fitness < best_fitness:
                    best_fitness = fitness
                    best_individual = individual

            generation_time = time.time() - start_time
            avg_fitness = np.mean(fitness_scores)

            print(f"\nGeneration {generation + 1} Results:")
            print(f"Best Fitness: {min(fitness_scores):.6f}")
            print(f"Average Fitness: {avg_fitness:.6f}")
            print(f"Time: {generation_time:.1f}s")

            # Store best individual from this generation
            gen_best_idx = fitness_scores.index(min(fitness_scores))
            self.best_individuals.append((population[gen_best_idx], min(fitness_scores)))

            # Early stopping if we're not improving
            if generation >= 3:
                recent_improvements = [self.best_individuals[i][1] for i in range(-3, 0)]
                if max(recent_improvements) - min(recent_improvements) < 0.0001:
                    print(f"\nEarly stopping: No significant improvement in last 3 generations")
                    break

            # Selection
            selected = self.selection(population, fitness_scores)

            # Create new population through crossover and mutation
            new_population = []
            for i in range(0, len(selected), 2):
                parent1 = selected[i]
                parent2 = selected[i + 1] if i + 1 < len(selected) else selected[0]

                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)

                new_population.extend([child1, child2])

            population = new_population[:self.population_size]

        print(f"\nGA Optimization Complete!")
        print(f"Best Fitness Achieved: {best_fitness:.6f}")
        return best_individual, best_fitness

In [51]:
class StockPredictor:
    """Stock price predictor using LSTM"""
    def __init__(self, symbol='AAPL', sequence_length=60, hidden_size=50, num_layers=2):
        self.symbol = symbol
        self.sequence_length = sequence_length
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.raw_data = None
        self.feature_cols = ['Close', 'Volume', 'High', 'Low', 'Open']

    def fetch_data(self, period='2y'):
        """Fetch stock data from Yahoo Finance"""
        print(f"Fetching data for {self.symbol}...")
        stock = yf.Ticker(self.symbol)
        data = stock.history(period=period)
        self.raw_data = data
        return data

    def prepare_data(self, data, feature_cols=['Close']):
        """Prepare data for training"""
        df = data[feature_cols].copy()

        # Handle NaN values
        if df.isna().sum().sum() > 0:
            print("NaNs detected! Applying forward fill...")
            df = df.ffill().bfill()

        scaled_data = self.scaler.fit_transform(df)

        X, y = [], []
        for i in range(self.sequence_length, len(scaled_data)):
            X.append(scaled_data[i-self.sequence_length:i])
            y.append(scaled_data[i, 0])  # Predict 'Close' price

        X, y = np.array(X), np.array(y)

        # Split into training and test sets
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        # Convert to PyTorch tensors
        X_train = torch.FloatTensor(X_train).to(self.device)
        y_train = torch.FloatTensor(y_train).to(self.device)
        X_test = torch.FloatTensor(X_test).to(self.device)
        y_test = torch.FloatTensor(y_test).to(self.device)

        return X_train, y_train, X_test, y_test, df.index[self.sequence_length:]

    def train_model(self, X_train, y_train, X_test, y_test, epochs=100, learning_rate=0.001):
        """Train the LSTM model"""
        input_size = X_train.shape[2]

        self.model = LSTM(
            input_size=input_size,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            output_size=1
        ).to(self.device)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        train_losses, test_losses = [], []
        print("Training model...")

        for epoch in range(epochs):
            # Training
            self.model.train()
            optimizer.zero_grad()
            train_pred, _ = self.model(X_train)
            train_loss = criterion(train_pred.squeeze(), y_train)
            train_loss.backward()
            optimizer.step()

            # Validation
            self.model.eval()
            with torch.no_grad():
                test_pred, _ = self.model(X_test)
                test_loss = criterion(test_pred.squeeze(), y_test)

            train_losses.append(train_loss.item())
            test_losses.append(test_loss.item())

            if (epoch + 1) % 20 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss.item():.6f}, Test Loss: {test_loss.item():.6f}")

        return train_losses, test_losses

    def predict(self, X_test):
        """Make predictions"""
        self.model.eval()
        with torch.no_grad():
            predictions, _ = self.model(X_test)
        return predictions.cpu().detach().numpy()

    def inverse_transform(self, scaled_data):
        """Inverse transform scaled predictions back to original scale"""
        dummy = np.zeros((len(scaled_data), self.scaler.n_features_in_))
        dummy[:, 0] = scaled_data.flatten()
        return self.scaler.inverse_transform(dummy)[:, 0]

    def evaluate_model(self, y_true, y_pred):
        """Evaluate model performance"""
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        return {'MSE': mse, 'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

    def plot_results(self, y_true, y_pred, dates, train_losses, test_losses):
        """Plot actual vs predicted prices and training history"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # Plot 1: Actual vs Predicted
        ax1.plot(dates, y_true, label='Actual', linewidth=2)
        ax1.plot(dates, y_pred, label='Predicted', linewidth=2, linestyle='--')
        ax1.set_title(f'{self.symbol} Stock Price Prediction', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Price ($)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Plot 2: Training and Validation Loss
        ax2.plot(train_losses, label='Training Loss', linewidth=2)
        ax2.plot(test_losses, label='Validation Loss', linewidth=2)
        ax2.set_title('Training History')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Loss')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

In [52]:
def get_company_selection():
    """Interactive company selection"""
    popular_stocks = {
        '1': ('AAPL', 'Apple Inc.'),
        '2': ('GOOGL', 'Alphabet Inc. (Google)'),
        '3': ('MSFT', 'Microsoft Corporation'),
        '4': ('AMZN', 'Amazon.com Inc.'),
        '5': ('TSLA', 'Tesla Inc.'),
        '6': ('META', 'Meta Platforms Inc. (Facebook)'),
        '7': ('NVDA', 'NVIDIA Corporation'),
        '8': ('NFLX', 'Netflix Inc.'),
        '9': ('JPM', 'JPMorgan Chase & Co.'),
        '10': ('JNJ', 'Johnson & Johnson'),
    }

    print("\n" + "="*45)
    print("GA-OPTIMIZED STOCK PRICE PREDICTION SYSTEM")
    print("="*45)
    print("\nSelect a company to predict:")
    print("-" * 40)
    for key, (symbol, name) in popular_stocks.items():
        print(f"{key:2}. {symbol:5} - {name}")
    print(f"{len(popular_stocks)+1:2}. Custom - Enter your own stock symbol")

    while True:
        choice = input(f"\nEnter your choice (1-{len(popular_stocks)+1}): ").strip()
        if choice in popular_stocks:
            symbol, name = popular_stocks[choice]
            print(f"\nSelected: {name} ({symbol})")
            return symbol, name
        elif choice == str(len(popular_stocks)+1):
            while True:
                symbol = input("Enter stock symbol (e.g., AAPL, GOOGL): ").strip().upper()
                if symbol and len(symbol) <= 6:
                    print(f"\n Selected: {symbol}")
                    return symbol, symbol
                print("Please enter a valid stock symbol")
        else:
            print(f"Please enter a number between 1 and {len(popular_stocks)+1}")

In [53]:
def predict_next_day_price(predictor, X_test, current_price):
    """Enhanced next day prediction with confidence indicators"""
    last_sequence = X_test[-1:].to(predictor.device)
    with torch.no_grad():
        future_pred, _ = predictor.model(last_sequence)

    predicted_price = predictor.inverse_transform(future_pred.cpu().detach().numpy())[0]
    price_change = predicted_price - current_price
    price_change_pct = (price_change / current_price) * 100

    if price_change > 0:
        trend = "📈 BULLISH"
    else:
        trend = "📉 BEARISH"

    return predicted_price, price_change, price_change_pct, trend

In [54]:
def display_ga_results(best_params, best_fitness):
    """Display GA optimization results"""
    print(f"\nGENETIC ALGORITHM OPTIMIZATION RESULTS")
    print("="*60)
    print(f"Best Hyperparameters Found:")
    print(f"Sequence Length: {best_params.sequence_length} days")
    print(f"Hidden Size: {best_params.hidden_size} units")
    print(f"Number of Layers: {best_params.num_layers}")
    print(f"Dropout Rate: {best_params.dropout:.3f}")
    print(f"Learning Rate: {best_params.learning_rate:.6f}")
    print(f"Batch Size: {best_params.batch_size}")
    print(f"Epochs: {best_params.epochs}")
    print(f"Best Validation Loss: {best_fitness:.6f}")
    print("="*60)

In [55]:
def display_prediction_summary(symbol, company_name, current_price, predicted_price,
                             price_change, price_change_pct, trend, metrics, ga_optimized=False):
    """Display comprehensive prediction summary"""
    optimization_text = "GA-OPTIMIZED " if ga_optimized else ""
    print(f"\n{'='*40}")
    print(f"{optimization_text}STOCK PREDICTION RESULTS")
    print("="*40)
    print(f"\nCOMPANY: {company_name} ({symbol})")
    print(f"CURRENT PRICE: ${current_price:.2f}")
    print(f"PREDICTED NEXT DAY PRICE: ${predicted_price:.2f}")
    print(f"EXPECTED CHANGE: ${price_change:+.2f} ({price_change_pct:+.2f}%)")
    print(f"TREND SIGNAL: {trend}")

    print(f"\nMODEL ACCURACY METRICS:")
    print(f"Mean Absolute Percentage Error: {metrics['MAPE']:.2f}%")
    print(f"Average Error: ${metrics['MAE']:.2f}")
    print(f"Root Mean Square Error: ${metrics['RMSE']:.2f}")

    # Confidence indicator based on MAPE
    if metrics['MAPE'] < 3.0:
        confidence = "HIGH"
    elif metrics['MAPE'] < 5.0:
        confidence = "MEDIUM"
    else:
        confidence = "LOW"

    print(f"Prediction Confidence: {confidence}")

    if ga_optimized:
        print(f"\n Hyperparameters optimized using Genetic Algorithm")

    print("\n" + "="*80)
    print("DISCLAIMER: This prediction is for educational purposes only.")
    print("Always consult financial advisors before making investment decisions.")
    print("="*80)

In [56]:
def main_with_ga():
    """Main function with GA optimization"""
    # Get user's company choice
    symbol, company_name = get_company_selection()

    print(f"\n{'='*60}")
    print(f"GA-OPTIMIZED STOCK PREDICTION FOR {company_name} ({symbol})")
    print("="*60)

    # Ask user if they want to use GA optimization
    print("\n Optimization Options:")
    print("1. Use Genetic Algorithm to find best hyperparameters (Recommended, ~10-15 minutes)")
    print("2. Use default hyperparameters (Fast, ~1 minute)")

    while True:
        choice = input("\nChoose optimization method (1 or 2): ").strip()
        if choice in ['1', '2']:
            break
        print("Please enter 1 or 2")

    use_ga = (choice == '1')

    # Initialize predictor with default parameters
    predictor = StockPredictor(
        symbol=symbol,
        sequence_length=60,
        hidden_size=100,
        num_layers=3
    )

    try:
        # Fetch and prepare data
        print(f"\nFetching data for {company_name}...")
        data = predictor.fetch_data(period='2y')
        print(f"Last data date: {data.index[-1].date()}")
        print(f"Predicting price for: {data.index[-1].date() + timedelta(days=1)}")

        current_price = data['Close'].iloc[-1]
        feature_cols = ['Close', 'Volume', 'High', 'Low', 'Open']
        predictor.feature_cols = feature_cols

        best_params = None
        if use_ga:
            # Initialize GA optimizer
            ga_optimizer = GeneticAlgorithmOptimizer(
                population_size=12,  # Smaller for faster execution
                generations=6,       # Fewer generations for demo
                mutation_rate=0.15,
                crossover_rate=0.8
            )

            # Run GA optimization
            best_params, best_fitness = ga_optimizer.optimize(predictor)
            display_ga_results(best_params, best_fitness)

            # Update predictor with optimized parameters
            predictor.sequence_length = best_params.sequence_length
            predictor.hidden_size = best_params.hidden_size
            predictor.num_layers = best_params.num_layers

        # Prepare data with final parameters
        print(f"\nPreparing data with {len(feature_cols)} features...")
        X_train, y_train, X_test, y_test, dates = predictor.prepare_data(data, feature_cols)
        print(f"Training data shape: {X_train.shape}")
        print(f"Test data shape: {X_test.shape}")

        # Train final model
        print(f"\nTraining final model for {company_name}...")
        if best_params and use_ga:
            train_losses, test_losses = predictor.train_model(
                X_train, y_train, X_test, y_test,
                epochs=best_params.epochs,
                learning_rate=best_params.learning_rate
            )
        else:
            train_losses, test_losses = predictor.train_model(
                X_train, y_train, X_test, y_test,
                epochs=100,
                learning_rate=0.001
            )

        # Make predictions on test data
        print("Making test predictions...")
        predictions = predictor.predict(X_test)

        # Inverse transform to get actual prices
        y_test_actual = predictor.inverse_transform(y_test.cpu().numpy())
        y_pred_actual = predictor.inverse_transform(predictions)

        # Evaluate model
        metrics = predictor.evaluate_model(y_test_actual, y_pred_actual)

        # Predict next day price
        predicted_price, price_change, price_change_pct, trend = predict_next_day_price(
            predictor, X_test, current_price
        )

        # Display results
        display_prediction_summary(
            symbol, company_name, current_price, predicted_price,
            price_change, price_change_pct, trend, metrics, ga_optimized=use_ga
        )

        print(f"\nPrediction completed for {company_name}!")
        if use_ga and best_params:
            print(f"\nFinal Optimized Configuration:")
            for param, value in best_params.to_dict().items():
                print(f"{param}: {value}")

        # Return results including predictions
        return predicted_price, metrics, best_params

    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        print("Try selecting a different stock symbol or check your internet connection.")
        return None, None, None, None, None

In [57]:
def quick_ga_predict(symbol, use_ga=True):
    """Quick GA-optimized prediction function for specific stocks"""
    print(f"\nQuick GA Prediction for {symbol}")
    print("-" * 40)

    predictor = StockPredictor(
        symbol=symbol,
        sequence_length=60,
        hidden_size=100,
        num_layers=3
    )

    try:
        data = predictor.fetch_data(period='2y')
        feature_cols = ['Close', 'Volume', 'High', 'Low', 'Open']
        predictor.feature_cols = feature_cols

        best_params = None
        if use_ga:
            # Quick GA optimization with smaller parameters
            ga_optimizer = GeneticAlgorithmOptimizer(
                population_size=8,
                generations=4,
                mutation_rate=0.2,
                crossover_rate=0.8
            )
            best_params, _ = ga_optimizer.optimize(predictor)

            # Update predictor
            predictor.sequence_length = best_params.sequence_length
            predictor.hidden_size = best_params.hidden_size
            predictor.num_layers = best_params.num_layers

        X_train, y_train, X_test, y_test, _ = predictor.prepare_data(data, feature_cols)

        # Train with optimized or default parameters
        if best_params:
            predictor.train_model(X_train, y_train, X_test, y_test,
                                epochs=min(best_params.epochs, 80),
                                learning_rate=best_params.learning_rate)
        else:
            predictor.train_model(X_train, y_train, X_test, y_test, epochs=80, learning_rate=0.001)

        predictions = predictor.predict(X_test)
        y_test_actual = predictor.inverse_transform(y_test.cpu().numpy())
        y_pred_actual = predictor.inverse_transform(predictions)
        metrics = predictor.evaluate_model(y_test_actual, y_pred_actual)

        current_price = data['Close'].iloc[-1]
        predicted_price, price_change, price_change_pct, trend = predict_next_day_price(
            predictor, X_test, current_price
        )

        optimization_text = "GA-Optimized " if use_ga else "Default"
        print(f"\n{symbol} {optimization_text}PREDICTION:")
        print(f"Current: ${current_price:.2f} → Predicted: ${predicted_price:.2f}")
        print(f"Change: ${price_change:+.2f} ({price_change_pct:+.2f}%) - {trend}")
        print(f"Model Accuracy: {metrics['MAPE']:.2f}% MAPE")
        if best_params:
            print(f"Best Config: seq_len={best_params.sequence_length}, hidden={best_params.hidden_size}, layers={best_params.num_layers}")

        return predicted_price, metrics, best_params

    except Exception as e:
        print(f"Error: {str(e)}")
        return None, None, None

In [58]:
def compare_optimization_methods(symbol):
    """Compare GA-optimized vs default hyperparameters"""
    print(f"\nCOMPARING OPTIMIZATION METHODS FOR {symbol}")
    print("="*60)

    # Test with default parameters
    print("\nTesting with DEFAULT hyperparameters...")
    default_pred, default_metrics, _ = quick_ga_predict(symbol, use_ga=False)

    # Test with GA optimization
    print("\nTesting with GA-OPTIMIZED hyperparameters...")
    ga_pred, ga_metrics, ga_params = quick_ga_predict(symbol, use_ga=True)

    if default_pred and ga_pred:
        print(f"\nCOMPARISON RESULTS:")
        print("-" * 40)
        print(f"Method          MAPE    RMSE     MAE")
        print("-" * 40)
        print(f"Default      {default_metrics['MAPE']:7.2f}% {default_metrics['RMSE']:7.2f} {default_metrics['MAE']:7.2f}")
        print(f"GA-Optimized {ga_metrics['MAPE']:7.2f}% {ga_metrics['RMSE']:7.2f} {ga_metrics['MAE']:7.2f}")
        print("-" * 40)

        improvement = default_metrics['MAPE'] - ga_metrics['MAPE']
        if improvement > 0:
            print(f"GA Optimization improved MAPE by {improvement:.2f}%")
        else:
            print(f"Default parameters performed better by {abs(improvement):.2f}%")

    return ga_params

In [59]:
if __name__ == "__main__":
    # Run the main GA-optimized prediction system
    print("Welcome to GA-Optimized LSTM Stock Prediction!")
    print("This system uses Genetic Algorithms to find optimal hyperparameters")
    

    choice = input("\n Choose mode:\n1. Interactive prediction with GA\n2. Quick comparison test\nEnter choise (1 or 2): ")

    if choice == '2':
        symbol = input("Enter stock symbol for comparison (e.g., AAPL): ").strip().upper()
        compare_optimization_methods(symbol)
    else:
        main_with_ga()

Welcome to GA-Optimized LSTM Stock Prediction!
This system uses Genetic Algorithms to find optimal hyperparameters



 Choose mode:
1. Interactive prediction with GA
2. Quick comparison test
Enter choise (1 or 2):  1



GA-OPTIMIZED STOCK PRICE PREDICTION SYSTEM

Select a company to predict:
----------------------------------------
1 . AAPL  - Apple Inc.
2 . GOOGL - Alphabet Inc. (Google)
3 . MSFT  - Microsoft Corporation
4 . AMZN  - Amazon.com Inc.
5 . TSLA  - Tesla Inc.
6 . META  - Meta Platforms Inc. (Facebook)
7 . NVDA  - NVIDIA Corporation
8 . NFLX  - Netflix Inc.
9 . JPM   - JPMorgan Chase & Co.
10. JNJ   - Johnson & Johnson
11. Custom - Enter your own stock symbol



Enter your choice (1-11):  4



Selected: Amazon.com Inc. (AMZN)

GA-OPTIMIZED STOCK PREDICTION FOR Amazon.com Inc. (AMZN)

 Optimization Options:
1. Use Genetic Algorithm to find best hyperparameters (Recommended, ~10-15 minutes)
2. Use default hyperparameters (Fast, ~1 minute)



Choose optimization method (1 or 2):  2



Fetching data for Amazon.com Inc....
Fetching data for AMZN...
Last data date: 2025-10-29
Predicting price for: 2025-10-30

Preparing data with 5 features...
Training data shape: torch.Size([353, 60, 5])
Test data shape: torch.Size([89, 60, 5])

Training final model for Amazon.com Inc....
Training model...
Epoch [20/100], Train Loss: 0.034116, Test Loss: 0.103115
Epoch [40/100], Train Loss: 0.013910, Test Loss: 0.021504
Epoch [60/100], Train Loss: 0.007035, Test Loss: 0.007388
Epoch [80/100], Train Loss: 0.007130, Test Loss: 0.004254
Epoch [100/100], Train Loss: 0.006049, Test Loss: 0.003973
Making test predictions...

STOCK PREDICTION RESULTS

COMPANY: Amazon.com Inc. (AMZN)
CURRENT PRICE: $230.30
PREDICTED NEXT DAY PRICE: $217.22
EXPECTED CHANGE: $-13.08 (-5.68%)
TREND SIGNAL: 📉 BEARISH

MODEL ACCURACY METRICS:
Mean Absolute Percentage Error: 2.55%
Average Error: $5.73
Root Mean Square Error: $6.89
Prediction Confidence: HIGH

DISCLAIMER: This prediction is for educational purposes 

In [60]:
if __name__ == "__main__":
    # Run the main GA-optimized prediction system
    print("Welcome to GA-Optimized LSTM Stock Prediction!")
    print("This system uses Genetic Algorithms to find optimal hyperparameters")
    

    choice = input("\n Choose mode:\n1. Interactive prediction with GA \n2. Quick comparison test \nEnter choice (1, or 2): ")

    if choice == '2':
        symbol = input("Enter stock symbol for comparison (e.g., AAPL): ").strip().upper()
        compare_optimization_methods(symbol)
    else:
        main_with_ga()

Welcome to GA-Optimized LSTM Stock Prediction!
This system uses Genetic Algorithms to find optimal hyperparameters



 Choose mode:
1. Interactive prediction with GA 
2. Quick comparison test 
Enter choice (1, or 2):  1



GA-OPTIMIZED STOCK PRICE PREDICTION SYSTEM

Select a company to predict:
----------------------------------------
1 . AAPL  - Apple Inc.
2 . GOOGL - Alphabet Inc. (Google)
3 . MSFT  - Microsoft Corporation
4 . AMZN  - Amazon.com Inc.
5 . TSLA  - Tesla Inc.
6 . META  - Meta Platforms Inc. (Facebook)
7 . NVDA  - NVIDIA Corporation
8 . NFLX  - Netflix Inc.
9 . JPM   - JPMorgan Chase & Co.
10. JNJ   - Johnson & Johnson
11. Custom - Enter your own stock symbol



Enter your choice (1-11):  4



Selected: Amazon.com Inc. (AMZN)

GA-OPTIMIZED STOCK PREDICTION FOR Amazon.com Inc. (AMZN)

 Optimization Options:
1. Use Genetic Algorithm to find best hyperparameters (Recommended, ~10-15 minutes)
2. Use default hyperparameters (Fast, ~1 minute)



Choose optimization method (1 or 2):  1



Fetching data for Amazon.com Inc....
Fetching data for AMZN...
Last data date: 2025-10-29
Predicting price for: 2025-10-30

Starting Genetic Algorithm Optimization
Population: 12, Generations: 6

Generation 1/6
----------------------------------------
Individual 1/12
Testing: seq_len=60, hidden=100, layers=3, lr=0.0010
→ Validation Loss: 0.002411
Individual 2/12
Testing: seq_len=30, hidden=128, layers=2, lr=0.0020
→ Validation Loss: 0.001692
Individual 3/12
Testing: seq_len=90, hidden=64, layers=4, lr=0.0008
→ Validation Loss: 0.002983
Individual 4/12
Testing: seq_len=80, hidden=142, layers=2, lr=0.0087
→ Validation Loss: 0.001399
Individual 5/12
Testing: seq_len=39, hidden=171, layers=4, lr=0.0010
→ Validation Loss: 0.002480
Individual 6/12
Testing: seq_len=84, hidden=63, layers=2, lr=0.0027
→ Validation Loss: 0.002204
Individual 7/12
Testing: seq_len=49, hidden=75, layers=3, lr=0.0041
→ Validation Loss: 0.001902
Individual 8/12
Testing: seq_len=45, hidden=169, layers=2, lr=0.0050
→ 